In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [2]:
policy_dict = {'Credit' : 'Credit', 'Direct' : 'Direct payment', 'Fine' : 'Fine', 'General' : 'Unknown', 
               'Guarantee' : 'Credit', 'Supplies' : 'Supplies', 'Tax' : 'Tax deduction', 
               'Technical' : 'Technical assistance', 'Unknown' : 'Unknown', 'Other' : 'Unknown', 'Nan' : 'Unknown' }

incentive_dict = {'Incentive' : 'Incentive', 'Disincentive' : 'Incentive', 'Unknown' : 'not_Incentive', 'Nan' : 'not_Incentive'}

In [82]:
rater = "Rater3"
# paths = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater2")
# paths = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater2")
paths = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/" + rater).glob('**/*.xlsx')

def clean_df(df, reference_column_to_drop_na):
    df.dropna(subset = [reference_column_to_drop_na], inplace = True)
    df.dropna(axis=1, how='all', inplace = True)
    df.replace(np.nan, 'Nan', regex=True, inplace = True)
    return df

def process_new_labels(filename, reference_column_to_drop_na, policy):
    df_temp = pd.concat(pd.read_excel(filename, engine='openpyxl', sheet_name = None), ignore_index = True)
    df_temp = clean_df(df_temp, reference_column_to_drop_na)
    df_temp.insert(0, "Document", df_temp.apply(lambda row: row.Sentence_Id.split("_")[0], axis = 1))
    df_temp.loc[df_temp['Is_policy'] == 0, 'Is_policy'] = "Nan"
    df_temp.loc[df_temp['Is_policy'] == 1, 'Is_policy'] = policy
    if "Other_instrument"in df_temp.columns:
        df_temp['Is_policy'] = np.where(df_temp['Is_policy'] == "Nan", df_temp['Other_instrument'], df_temp['Is_policy'])
    df_temp.loc[df_temp['Is_incentive'] == 0, 'Is_incentive'] = "Unknown"
    df_temp.loc[df_temp['Is_incentive'] == 1, 'Is_incentive'] = "Incentive"
    return df_temp


results_list = []
for path in paths:
    # because path is object not string
    path_in_str = str(path)
    print(path_in_str)
    flag = False
    for policy in policy_dict:
        if policy in path_in_str:
            policy_instrument = policy
            flag = True
    if flag:
        df = process_new_labels(path_in_str, "Is_incentive", policy_instrument)
        results_list.append(df[["Document", "Sentence_Id", "Sentence", "Is_policy", "Is_incentive"]].values.tolist())
print("There are:", len(results_list), "results")
# data = df[["Document", "Sentence", "Primary_Instrument", "Category"]].to_numpy()
# filename = "WRI_Policy_Tags.xlsx"
# file = paths / filename
# df = pd.concat(pd.read_excel(file, engine='openpyxl', sheet_name = None), ignore_index = True)


C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater3\Unique_sentence_IDs_Credit.xlsx
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater3\Unique_sentence_IDs_Direct_payment.xlsx
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater3\Unique_sentence_IDs_Fine.xlsx
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater3\Unique_sentence_IDs_Tax_deduction.xlsx
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater3\Unique_sentence_IDs_Technical_assistance.xlsx
C:\Users\user\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater3\WRI_Policy_Tags.xlsx
There are: 5 results


In [83]:
def label_cleaning(dictionaryionary, label):
    flag = 1
    
    for key in dictionaryionary:
#         print(key, "----", label)
        if key in label:
            return dictionaryionary[key]
            flag = 0
            break
    if flag == 1:
#         print(label)
        return label

dictionary = {}

i = 0
for item1 in results_list:
    for item in item1:
        if item[0] in dictionary:
            dictionary[item[0]]["Seccion unica"]["sentences"][item[1]] = {"text" : item[2], "labels" : label_cleaning(policy_dict, item [3]), "incentive": label_cleaning(incentive_dict, item [4])}
        else:
            dictionary[item[0]] = {"Seccion unica" : {"tags" : [], "sentences" : {}}}
            dictionary[item[0]]["Seccion unica"]["sentences"][item[1]] = {"text" : item[2], "labels" : label_cleaning(policy_dict, item [3]), "incentive": label_cleaning(incentive_dict, item [4])}


In [84]:
path = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater3")
filename = "WRI_Policy_Tags.xlsx"
file = path / filename
df = pd.concat(pd.read_excel(file, engine='openpyxl', sheet_name = None), ignore_index = True)

df = clean_df(df, "Document")
data = df[["Document", "Sentence", "Primary_Instrument", "Category"]].to_numpy()

In [85]:
print(df.shape)
df.head()

(687, 8)


,Document,Sentence,Primary_Instrument,Other_Instrument,Land_Use_Type,Category,Unique_Policy,Key_words
0,2019 ACUERDO por el que se emiten los Lineamie...,Generar empleo y garantizara la población camp...,Technical assistance,"Supplies, Credit, Direct payment","Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),"insumo, crédito, capacitación, asistencia técnica"
1,2019 ACUERDO por el que se emiten los Lineamie...,El Programa incentivará a los sujetos agrarios...,General incentive,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),incentivar
2,2019 ACUERDO por el que se emiten los Lineamie...,Los sujetos agrarios beneficiados por el progr...,Supplies,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),"apoyo económico, apoyos en especie, insumos, h..."
3,2019 ACUERDO por el que se emiten los Lineamie...,"Adicionalmente, contarán con acompañamiento té...",Technical assistance,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),Nan
4,2019 ACUERDO por el que se emiten los Lineamie...,"El sujeto de derecho, recibirá un apoyo económ...",Direct payment,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),pesos


In [86]:
i = 0
for item in data:
    if item[0] in dictionary:
        i += 1
        dictionary[item[0]]["Seccion unica"]["sentences"][str(i)] = {"text" : item[1], "labels" : label_cleaning(policy_dict, item [2]), "incentive": label_cleaning(incentive_dict, item [3])}
    else:
        i += 1
        dictionary[item[0]] = {"Seccion unica" : {"tags" : [], "sentences" : {}}}
        dictionary[item[0]]["Seccion unica"]["sentences"][str(i)] = {"text" : item[1], "labels" : label_cleaning(policy_dict, item [2]), "incentive": label_cleaning(incentive_dict, item [3])}

In [87]:
print(len(dictionary))
dictionary

401


{'43f0373': {'Seccion unica': {'tags': [],
   'sentences': {'43f0373_23': {'text': 'Apoyar tecnicamente la ejecucion de programas y proyectos orientados al fortalecimiento de la competitividad de las micro, pequeñas y medianas empresas, especialmente en lo relacionado con los temas de calidad, productividad, innovacion o desarrollo tecnologico',
     'labels': 'Technical assistance',
     'incentive': 'Incentive'},
    '43f0373_20': {'text': 'Desarrollar una cultura de calidad en las actividades de produccion de bienes y la prestacion de servicios, asi como su mejora continua y promocion de la innovacion;',
     'labels': 'Unknown',
     'incentive': 'not_Incentive'},
    '43f0373_38': {'text': 'El Consejo Nacional de Calidad se relacionara con el Organo Ejecutivo a traves del Ministerio de Economia',
     'labels': 'Unknown',
     'incentive': 'not_Incentive'},
    '43f0373_104': {'text': 'Gozaran de autonomia financiera de acuerdo a lo establecido en la presente ley;',
     'labels':

In [88]:
sentence_dict = {}
i = 0
for item, value in dictionary.items():
    for key, sentences in value["Seccion unica"]["sentences"].items():
        i += 1
        sentence_dict[sentences["text"]] = [sentences["labels"], sentences["incentive"]]
print(i)
        

1526


In [89]:
sentence_dict

{'Apoyar tecnicamente la ejecucion de programas y proyectos orientados al fortalecimiento de la competitividad de las micro, pequeñas y medianas empresas, especialmente en lo relacionado con los temas de calidad, productividad, innovacion o desarrollo tecnologico': ['Technical assistance',
  'Incentive'],
 'Desarrollar una cultura de calidad en las actividades de produccion de bienes y la prestacion de servicios, asi como su mejora continua y promocion de la innovacion;': ['Unknown',
  'not_Incentive'],
 'El Consejo Nacional de Calidad se relacionara con el Organo Ejecutivo a traves del Ministerio de Economia': ['Unknown',
  'not_Incentive'],
 'Gozaran de autonomia financiera de acuerdo a lo establecido en la presente ley;': ['Unknown',
  'not_Incentive'],
 'Fomentar la aplicacion de las normas tecnicas en los distintos sectores productivos;': ['Technical assistance',
  'Incentive'],
 '- Focalizar, en mejor forma, los incentivos especiales a favor de los pequeños productores beneficiar

In [90]:
path = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/{}".format(rater))
filename = "{}_labeled.json".format(rater)
file = path / filename
with open(file, 'w') as fp:
    json.dump(dictionary, fp)

In [91]:
path = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/{}".format(rater))
filename = "{}_labeled_sentences.json".format(rater)
file = path / filename
with open(file, 'w') as fp:
    json.dump(sentence_dict, fp)